# Automation of imaging nuclei midplanes using **autoSTED**

We used the code in this notebook for overview-detail imaging of DNA-stained nuclei. The pipeline defined here will image confocal overview stacks, followed by STED midplane detail acquisitions of nuclei detected in the overviews (with on-the-fly stirching)

This was done to study changes in chromatin textured during (induced) replicative senescense for [Palikyras et al. (Aging Cell, 2024)](https://onlinelibrary.wiley.com/doi/10.1111/acel.14083)

### Old versions

**Note:** We used older (and messier) versions (v1.0.0) of autosted (then pipeline2) for most of the published studies, which can be found under: https://doi.org/10.5281/zenodo.14627119

The corresponding file in the v1.0.0 release is ```auto-nucleus-midplane.ipynb```

## Imports and plot settings

Run this once

In [5]:
import json
import logging
import os
import sys

import specpy as sp
from matplotlib import pyplot as plt

from autosted import AcquisitionPipeline
from autosted.callback_buildingblocks import (
    JSONSettingsLoader,
    LocationKeeper,
    LocationRemover,
    NewestSettingsSelector,
    ScanFieldSettingsGenerator,
    SpiralOffsetGenerator,
    StageOffsetsSettingsGenerator,
    StitchedNewestDataSelector,
)
from autosted.detection import SimpleFocusPlaneDetector
from autosted.detection.legacy import SimpleNucleusMidplaneDetector
from autosted.imspector import get_current_stage_coords
from autosted.stoppingcriteria import TimedStoppingCriterion
from autosted.task_filtering import AlreadyImagedFOVFilter
from autosted.taskgeneration import AcquisitionTaskGenerator

# plotting params
plt.rcParams["figure.figsize"] = [7, 7]

# configure logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Code definition for a single acquisition

Run this once

In [ ]:
def run_pipeline(
    save_path,
    ov_json,
    det_jsons,
    start_coords,
    ov_moves,
    hours_to_image=12,
    ignore_z_overlap_check=True,
    manual_z_offset=0,
):

    pl = AcquisitionPipeline(
        data_save_path=save_path,
        hierarchy_levels=("overview", "detail"),
        save_combined_hdf5=True,
    )

    pl.add_stopping_condition(TimedStoppingCriterion(hours_to_image * 60 * 60))

    # overview task generator: same settings every time
    atg_overview = AcquisitionTaskGenerator(
        "overview",
        LocationRemover(JSONSettingsLoader([ov_json])),
        StageOffsetsSettingsGenerator(
            SpiralOffsetGenerator(ov_moves, start_coords, return_parameter_dict=False)
        ),
        StageOffsetsSettingsGenerator(SimpleFocusPlaneDetector()),
    )

    # init detector in overview
    detector = SimpleNucleusMidplaneDetector(
        StitchedNewestDataSelector(pl, "overview"),
        n_classes=2,
        manual_offset=manual_z_offset,
        verbose=True,
        plot_detections=True,
        region_filters={"area": (100, 100000)},
        fov_expansion_factor=1.4,
    )

    # detail task generation: settings from last overview + new FOV from detector
    atg_detail = AcquisitionTaskGenerator(
        "detail",
        LocationKeeper(NewestSettingsSelector(pl, "overview")),
        LocationRemover(JSONSettingsLoader(det_jsons, None, False)),
        ScanFieldSettingsGenerator(detector),
    )

    # add filter to ignore new bounding boxes that have intersection-over-union >0.5
    # with any already scanned bounding boxes
    flt = AlreadyImagedFOVFilter(pl, "detail", 0.5, ignore_z_overlap_check)
    atg_detail.add_task_filters(flt)

    # overview callback: re-add overview
    pl.add_callback(atg_overview, "overview")
    # overview callback 2: detect & do detail
    pl.add_callback(atg_detail, "overview")

    # GO
    pl.run(atg_overview)

## 1) initialize acquisition list and connect to Imspector

Run this to initialize the acquisition queue and connect to imspector
Re-run to clear queued acquisitions

In [7]:
# init acq. list
acquisitions = []

# imspector object
im = sp.get_application()

## 2) Enqueue an acquisition

Set the parameters for a new acquisition and add it to the queue.
**Acquisition will be run at the current stage coordinates**

In [ ]:
params = {}

### CHANGE ME!
params["save_path"] = "acquisition_data/auto_nuclei"

# MANUAL OFFSET IN Z, e.g. to image bottom of cell
# params['manual_z_offset'] = 1e-6

##### (OPTIONALLY) CHANGE US!

# how long to image
params["hours_to_image"] = 24

### PARAMETER FILES
# paths of the parameters files
# we can use multiple for the STED measurement (e.g. to do both a 2d and 3d STED acq.)
params["ov_json"] = "config_json/test_overview.json"
params["det_jsons"] = [
    "config_json/test_detail.json",
    # 'config_json/test_detail_2.json'
]

### SPIRAL MOVE SIZE ###
# Set this smaller than the overview FOV if you want to use Stitching
params["ov_moves"] = [4e-5, 4e-5]  # how much to move in overview spiral

# whether to ignore the z coordinate in checking for already imaged regions
params["ignore_z_overlap_check"] = True

### Things that are set automatically
# ensure we use slashes and not backslashes
params["save_path"] = params["save_path"].replace(os.sep, "/")
# start at current coordinates, do not change!
params["start_coords"] = get_current_stage_coords(im)

# add to queue
acquisitions.append(params)

# print the currently queued acquisitions
print(
    """
Queued Acquisitions:
====================
"""
)
for ac in acquisitions:
    print(json.dumps(ac, indent=1))

## 3) Run the acquisitions

Execute cell below to run the enqueued acquisitions

In [ ]:
# go through queued acquisitions (in reverse)
# and run them
for ac in reversed(acquisitions):
    run_pipeline(**ac)

# Reset queued acquisitions
# NOTE: if you cancelled a run, this might not be executed,
# make sure to clear old acquisitions manually (step 1)
acquisitions = []